<a href="https://colab.research.google.com/github/jeekyung/ESAA/blob/main/ESAA_OB_week3_%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%99%84%EB%B2%BD_%EA%B0%80%EC%9D%B4%EB%93%9C_ch8_4_pg_487~496.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **04. 텍스트 분류 실습 - 20 뉴스그룹 분류**
---

>* 사이킷런이 내부에 가지고 있는 예제 데이터
* 텍스트 분류는 특정 문서의 분류를 학습 데이터를 통해 학습해 모델을 생성한 뒤 이 학습 모델을 이용해 다른 문서의 분류를 예측하는 것

###**텍스트 정규화**

In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data=fetch_20newsgroups(subset='all',random_state=156)

In [2]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


>filenames: 로컬 컴퓨터에 저장하는 디렉터리와 파일명 지칭

In [3]:
# target 클래스 구성 확인
import pandas as pd

print('target 클래스의 값과 분포도 \n',pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n',news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


>target 클래스의 값은 0부터 19까지 20개(target값 0:alt.athetism, target값 1:comp.graphics,)

In [4]:
#개별 데이터가 텍스트로 어떻게 구성돼있는지 확인
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

>* fetch_20newsgroups()는 subset 파라미터를 이용해 학습 테스트 데이터 세트를 분리해 내려받을 수 있음

In [5]:
from sklearn.datasets import fetch_20newsgroups

# subset = 'train'으로 학습용 데이터만 추출, remove=('headers','footers','quotes')로 내용만 추출
train_news=fetch_20newsgroups(subset='train',remove=('headers','footers','quotes'),
                   random_state=156)
X_train = train_news.data
y_train = train_news.target

#subset='test'으로 테스트 데이터만 추출, remove=('headers','footers','quoters')로 내용만 추출
test_news=fetch_20newsgroups(subset='test',remove=('headers','footers','quotes'),
                   random_state=156)
X_test = test_news.data
y_test = test_news.target
print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data),len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


###**피처 벡터화 변환과 머신러닝 모델 학습/예측/평가**

>* CountVectorizer 이용하여 학습 데이터의 텍스트를 피처 벡터화.
* 테스트 데이터에도 피처 벡터화를 수행(학습 데이터를 이용해 fit()수행된 CountVectorizer 객체로 변환(transform))

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

#Count Vectorization으로 피처 벡터화 변환 수행.
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

#학습 데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피처 벡터화 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 CountVectorizer Shape:',X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape: (11314, 101631)


>학습 데이터를 countVectorizer로 피처를 추출한 결과, 11314개의 문서에서 피처, 즉 단어가 101631개 만들어짐

In [7]:
#로지스틱 회귀 적용해 뉴스그룹에 대한 분류 예측
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect,y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.608


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


>count 기반으로 피처 벡터화가 적용된 데이터 세트에 대한 로지스틱 회귀의 예측 정확도는 약 0.608

In [8]:
# Count 기반에서 TF-IDF 기반으로 벡터화를 변경해서 예측 모델 수행
from sklearn.feature_extraction.text import TfidfVectorizer

#TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

In [9]:
#LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.674


>TF-IDF가 단순 카운트 기반보다 훨씬 높은 예측 정확도 제공함

>텍스트 분석에서 머신러닝 모델의 성능을 향상시키는 중요한 2가지 방법은<br/> 최적의 ML 알고리즘을 선택하는 것과 최상의 피처 전처리를 수행하는 것.

>좀 더 다양한 파라미터 적용. TfidfVectorizer 클래스의 스톱 워드를 기존 'None'에서 'english'로 변경하고, ngram_range는 (1,1)에서 (1,2)로, max_df=300으로 변경

In [10]:
#stop words 필터링을 추가하고 ngram을 기본 (1,1)dptj (1,2)로 변경해 피처 벡터화 적용.
tfidf_vect = TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.692


>이번에는 GridSearchCV 이용해 로지스틱 회귀의 하이퍼 파라미터 최적화 수행.<br/>
로지스틱 회귀의 C 파라미터만 변경하면서 최적의 C 값을 찾은 뒤 이 C값으로 학습된 모델에서 테스트 데이터로 예측해 성능 평가

In [11]:
from sklearn.model_selection import GridSearchCV

#최적 C 값 도출 튜닝 수행. CV는 3 폴드 세트로 설정.
params = {'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf,param_grid=params,cv=3,scoring='accuracy',verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect, y_train)
print('Logistic Regression best C parameter:',grid_cv_lr.best_params_)

#최적 C 값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Logistic Regression best C parameter: {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.701


>로지스틱 회귀의 C가 10일 때GridSearchCV의 교차 검증 테스트 세트에서 가장 좋은 예측 성능을 나타냈으며, 테스트 세트에 적용해 더 좋은 성능 수치 얻음

###**사이킷런 파이프라인 사용 및 GridSearchCV와의 결합**

In [12]:
from sklearn.pipeline import Pipeline

#TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성
pipeline = Pipeline([
    ('tfidf_vect',TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=300)),
    ('lr_clf',LogisticRegression(C=10))
])

#별도의 TfidfVectorizer 객체의 fit(), transform()과 LogisticRegression의 fit(), predict()가 필요 없음
#pipeline의 fit()과 predict()만으로 한꺼번에 피처 벡터화와 ML 학습/예측이 가능
pipeline.fit(X_train,y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline을 통한 Logistic Regression의 예측 정확도는 0.701


In [13]:
#Pipeline + GridSearchCV 적용(튜닝 시간 오래 소요됨)
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect',TfidfVectorizer(stop_words='english')),
    ('lr_clf',LogisticRegression())
])

#Pipeline에 기술된 각각의 객체 변수에 언더바(_) 2개를 연달아 붙여 GridSearchCV에 사용될 파라미터/하이퍼 파라미터 이름과 값 설정
params = {'tfidf_vect_ngram_range': [(1,1),(1,2),(1,3)],
          'tfidf_vect_max_df':[100,300,700],
          'lr_clf_C':[1,5,10]
}

#GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy',verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predic(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:3f}'.format(
    accuracy_score(y_test, pred)
))

Fitting 3 folds for each of 27 candidates, totalling 81 fits


ValueError: ignored

>정확도가 크게 개선되지는 않음.